In [6]:
import pybcclean as pbc
import igl
import meshplot as mp
import numpy as np
import networkx as nx
import os
import time
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pbc_utilities as pbcut
def separate_surfaces(v, f, fl):
    """
    :param v vertices
    :param f faces
    :param fl face labels
    perturb the surfaces according to the face labels
    :return v_dict, f_dict
    """
    v_dict = {}
    f_dict_temp = {}
    f_dict = {}
    count_dict = {}
    for fidx, lb in enumerate(fl):
        if lb not in f_dict_temp:
            f_dict_temp[lb] = np.zeros_like(f)
            f_dict_temp[lb][0,:]=f[fidx,:].copy()
            count_dict[lb] = 1
        else:
            f_dict_temp[lb][count_dict[lb],:]=f[fidx,:].copy()
            count_dict[lb] += 1
    for lb in f_dict_temp:
        f_dict_temp[lb] = f_dict_temp[lb][0:count_dict[lb],:]
        v_dict[lb], f_dict[lb], _, _= igl.remove_unreferenced(v,f_dict_temp[lb])
    return v_dict, f_dict

def perturb_and_union(v_dict, f_dict, eps):
    """
    :eps float to control the perturbation
    return new_v, new_f, new_fl
    """
    nv = 0 
    nf = 0
    for lb in f_dict:
        nv += len(v_dict[lb])
        nf += len(f_dict[lb])
    new_v = np.zeros((nv,3),dtype=float)
    new_f = np.zeros((nf,3), dtype=int)
    new_fl = np.zeros((nf,1), dtype=int)
    count_v = 0
    count_f =0
    for lb in f_dict:
        f_temp = f_dict[lb].copy()
        v_temp = v_dict[lb].copy()
        new_v[count_v:count_v+len(v_temp)]= v_temp + np.ones_like(v_temp) * np.random.uniform(-eps, eps)
        f_temp += np.ones_like(f_temp) * count_v
        count_v += len(v_temp)
        new_f[count_f:count_f+len(f_temp)]=f_temp
        new_fl[count_f:count_f+len(f_temp)]= np.ones((len(f_temp),1), dtype=int) *lb
        count_f += len(f_temp)
    return new_v, new_f, new_fl

In [26]:
parent_dir = "../data/14"
bad_obj_file = pbcut.get_bad_mesh(parent_dir)
dmat_file = parent_dir+"/feat.dmat"
v_bad, f_bad=igl.read_triangle_mesh(bad_obj_file)
fl_bad  = igl.read_dmat(dmat_file)
eps = 1
v_dict, f_dict = separate_surfaces(v_bad, f_bad, fl_bad)
new_v, new_f, new_fl = perturb_and_union(v_dict, f_dict, eps )
igl.write_obj("test"+str(eps)+".obj",new_v, new_f)
mp.plot(new_v, new_f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.603677…

True

In [27]:
plane_v, plane_f=igl.read_triangle_mesh("test_examplesairplane_0627_opt_HR.obj")

In [28]:
igl.is_edge_manifold(plane_f)

True

In [ ]:
igl.